# Section 1 - Pulling data from API Endpoints and Cleaning
Libraries

In [1]:
import os
import pandas as pd
import requests
from sqlalchemy import create_engine
from sqlalchemy import exc
from dotenv import load_dotenv

In [2]:
load_dotenv()

True

Set up credentials

In [3]:
auth_url = 'https://id.twitch.tv/oauth2/token'
cred = {'client_id': os.getenv('CLIENT_ID'),
           'client_secret': os.getenv('CLIENT_SECRET'),
           'grant_type':'client_credentials'}
r = requests.post(auth_url, data=cred)
data = r.json()
print('Bearer '+data['access_token'])

base_url = 'https://api.igdb.com/v4'
header = {'Client-ID': os.getenv('CLIENT_ID'),
           'Authorization': 'Bearer '+data['access_token']}

Bearer m7twd7xeh0e1vixmqrqg349ez2m4ao


## Games Endpoint
### games table
Set max games we will pull

In [48]:
max_games = 5000

In [61]:
i = 0
games_table_df = pd.DataFrame()
while i < max_games/500:
    read = '''fields name, category, first_release_date, status,
            aggregated_rating, aggregated_rating_count, rating, rating_count,
            total_rating, total_rating_count; where category = 0; limit 500; offset '''
    offset = str(500*i)
    semi = ';'
    data = read + offset + semi
    r = requests.post(base_url+'/games', headers=header,
    data= data)
    games_table = r.json()
    games_table_temp = pd.DataFrame(games_table)
    games_table_df=pd.concat([games_table_temp,games_table_df])
    i +=1

### genres table

In [49]:
i = 0
genres_table_df = pd.DataFrame()
while i < max_games/500:
    read = 'fields genres; limit 500; offset '
    offset = str(500*i)
    semi = ';'
    data = read + offset + semi
    r = requests.post(base_url+'/games', headers=header,
    data= data)
    genres_table = r.json()
    genres_table_temp =pd.DataFrame(genres_table).explode('genres')
    genres_table_df=pd.concat([genres_table_df,genres_table_temp])
    i +=1

In [51]:
genres_table_df

,id,genres
0,35004,15
0,35004,32
1,176043,10
2,89616,9
2,89616,33
...,...,...
495,93138,13
496,3371,10
497,55009,NaN
498,93964,NaN


### keywords table

In [70]:
i = 0
keywords_table_df = pd.DataFrame()
while i < max_games/500:
    read = 'fields keywords; limit 500; offset '
    offset = str(500*i)
    semi = ';'
    data = read + offset + semi
    r = requests.post(base_url+'/games', headers=header,
    data= data)
    keywords_table = r.json()
    keywords_table_temp =pd.DataFrame(keywords_table).explode('keywords')
    keywords_table_df=pd.concat([keywords_table_df,keywords_table_temp])
    i +=1

In [71]:
keywords_table_df

,id,keywords
0,35004,NaN
1,176043,5340
1,176043,23861
2,89616,NaN
3,118008,NaN
...,...,...
496,3371,28703
497,55009,970
497,55009,1026
498,93964,NaN


### platforms table

In [7]:
r = requests.post(base_url+'/games', headers=header,
data='fields platforms; limit 100;')
platforms_table = r.json()
platforms_table_df =pd.DataFrame(platforms_table).explode('platforms')
platforms_table_df

,id,platforms
0,35004,6
1,176043,87
2,89616,34
2,89616,39
3,118008,6
...,...,...
95,123238,6
96,196510,NaN
97,88217,NaN
98,209902,6


In [72]:
i = 0
platforms_table_df = pd.DataFrame()
while i < max_games/500:
    read = 'fields platforms; limit 500; offset '
    offset = str(500*i)
    semi = ';'
    data = read + offset + semi
    r = requests.post(base_url+'/games', headers=header,
    data= data)
    platforms_table = r.json()
    platforms_table_temp =pd.DataFrame(platforms_table).explode('platforms')
    platforms_table_df=pd.concat([platforms_table_df,platforms_table_temp])
    i +=1

In [73]:
platforms_table_df

,id,platforms
0,35004,6
1,176043,87
2,89616,34
2,89616,39
3,118008,6
...,...,...
496,3371,4
496,3371,7
497,55009,38
498,93964,NaN


## Genres Endpoint
### genres_info table

In [75]:
r = requests.post(base_url+'/genres', headers=header, data='fields name; limit 500;')
genre_info_table = r.json()
genre_info_table_df = pd.DataFrame(genre_info_table)
genre_info_table_df

,id,name
0,4,Fighting
1,5,Shooter
2,7,Music
3,8,Platform
4,9,Puzzle
5,10,Racing
6,11,Real Time Strategy (RTS)
7,12,Role-playing (RPG)
8,13,Simulator
9,14,Sport


## Keyword Endpoint
### keyword_info_table

In [89]:
max_keywords = 10_000
i = 0
keywords_info_table_df = pd.DataFrame()
while i < max_keywords/500:
    read = 'fields slug; limit 500; offset '
    offset = str(500*i)
    semi = ';'
    data = read + offset + semi
    r = requests.post(base_url+'/keywords', headers=header,
    data= data)
    keywords_info_table = r.json()
    keywords_info_table_temp =pd.DataFrame(keywords_info_table)
    keywords_info_table_df=pd.concat([keywords_info_table_df,keywords_info_table_temp])
    i +=1

In [90]:
keywords_info_table_df

,id,slug
0,3576,oi
1,4010,argentina
2,13017,fremen
3,1027,adventure
4,19226,fight-trivia
...,...,...
495,14655,big-bertha
496,20275,skydive-proximity-flight-game
497,20705,bentleys-hackpack-game
498,9758,3-d-glasses


## Platform Endpoint
### Platform Table

In [92]:
r = requests.post(base_url+'/platforms', headers=header, data='fields name; limit 500;')
platform_info_table = r.json()
platform_info_table_df = pd.DataFrame(platform_info_table)
platform_info_table_df

,id,name
0,158,Commodore CDTV
1,339,Sega Pico
2,8,PlayStation 2
3,39,iOS
4,94,Commodore Plus/4
...,...,...
194,440,Visual Memory Unit / Visual Memory System
195,441,PocketStation
196,29,Sega Mega Drive/Genesis
197,390,PlayStation VR2


# Section 2 - Clean and Format Data

In [11]:
# Renaming df tables to sync with DB Schema
# Games Table:
games_table_df = games_table_df.rename(columns={"id":"game_id"})

# Genre Tables:
genres_table_df = genres_table_df.rename(columns={"id":"game_id", "genres":"genre_id"})
genre_info_table_df = genre_info_table_df.rename(columns={"id":"genre_id", "name":"genre_name"})

# Keyword Tables:
keywords_table_df = keywords_table_df.rename(columns={"id":"game_id", "keywords":"keyword_id"})
keyword_info_table_df = keyword_info_table_df.rename(columns={"id":"keyword_id", "slug":"keyword_name"})

# Platform Tables:
platforms_table_df = platforms_table_df.rename(columns={"id":"game_id", "platforms":"platform_id"})
platform_info_table_df = platform_info_table_df.rename(columns={"id":"platform_id", "name":"platform_name"})

In [12]:
# Converting unix time to datetime format
games_table_df['first_release_date'] = pd.to_datetime(games_table_df['first_release_date'], unit='s', origin='unix')

# Section 3 - Upload to AWS RDS MySQL Server

In [13]:
engine = create_engine(f"mysql+pymysql://{os.getenv('USER')}:{os.getenv('PASS')}@{os.getenv('RDS_ENDPOINT')}/igdb"
                       .format(host= os.getenv("RDS_ENDPOINT"),
                               port= os.getenv("RDS_PORT"),
                               user=os.getenv('USER'),
                               pw=os.getenv('PASS'),
                               ))

In [15]:
for i in range(len(games_table_df)):
    try:
        games_table_df.iloc[i:i+1].to_sql(name="games",if_exists='append',con = engine, index=False)
    except exc.IntegrityError as e:
        pass

In [16]:
for i in range(len(genres_table_df)):
    try:
        genres_table_df.iloc[i:i+1].to_sql(name="genres",if_exists='append',con = engine, index=False)
    except exc.IntegrityError as e:
        pass

In [17]:
for i in range(len(keywords_table_df)):
    try:
        keywords_table_df.iloc[i:i+1].to_sql(name="keywords",if_exists='append',con = engine, index=False)
    except exc.IntegrityError as e:
        pass

In [18]:
for i in range(len(platforms_table_df)):
    try:
        platforms_table_df.iloc[i:i+1].to_sql(name="platforms",if_exists='append',con = engine, index=False)
    except exc.IntegrityError as e:
        pass

In [19]:
genre_info_table_df.to_sql(name="genres_info",if_exists='replace', con = engine, index=False)
keyword_info_table_df.to_sql(name='keyword_info',if_exists='replace', con= engine, index=False)
platform_info_table_df.to_sql(name="platform_info", if_exists='replace', con= engine, index=False)

100

# Section 4 - Example Queries
## Top Rated Tables by Game Rating

In [ ]:
# Genres with top rated games
SELECT gni.genre_name, AVG(gm.total_rating)
FROM games AS gm
INNER JOIN genres AS gn ON gm.game_id = gn.game_id
INNER JOIN genres_info AS gni ON gn.genre_id = gni.genre_id
GROUP BY gni.genre_id;

In [ ]:
SELECT ki.keyword_name, AVG(gm.rating)
FROM games AS gm
INNER JOIN keywords AS k ON gm.game_id = k.game_id
INNER JOIN keyword_info AS ki ON k.keyword_id = ki.keyword_id
GROUP BY ki.keyword_id;

In [ ]:
SELECT pi.platform_name, AVG(gm.rating)
FROM games AS gm
INNER JOIN platforms AS p ON gm.game_id = p.game_id
INNER JOIN platform_info AS pi ON p.platform_id = pi.platform_id
GROUP BY pi.platform_id;